In [5]:
from selenium import webdriver
from tempfile import mkdtemp
import sqlalchemy
import requests
import numpy as np
import pandas as pd
import datetime
import json
import urllib
from sqlalchemy import create_engine
from selenium_stealth import stealth
import pymysql
import undetected_chromedriver as uc
from sqlalchemy.engine.url import URL
from selenium.webdriver.common.by import By


URL_SEARCH = "https://www.tickpick.com/concerts/yeat-tickets/"

options = uc.ChromeOptions()
#options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument("--disable-gpu")
# options.add_argument("--window-size=1280x1696")
# options.add_argument("--single-process")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-dev-tools")
# options.add_argument("--no-zygote")
# options.add_argument(f"--user-data-dir={mkdtemp()}")
# options.add_argument(f"--data-path={mkdtemp()}")
# options.add_argument(f"--disk-cache-dir={mkdtemp()}")
# options.add_argument("--remote-debugging-port=9222")

#driver = webdriver.Chrome(r"C:\Users\Paul\Documents\chromedriver.exe", options=options)
driver = uc.Chrome(options=options)


driver.get(URL_SEARCH)

xpath_links = "//*[@id=\"events\"]/div[*]/a[1]"

links = []
while(not(links)):
    links = driver.find_elements_by_xpath(xpath_links)

venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")

links = list(map(lambda elem: elem.get_attribute("href"), links))
venue_names = list(map(lambda elem: elem.text, venue_names))
dates = list(map(lambda elem: elem.text, dates))
print(links)

def add_page_to_row_list(link, row_list, driver, venue_name, date, time):
    if(link == 'https://www.tickpick.com/buy-yeat-tickets-newport-music-hall-4-9-22-7pm/5076846/'):
        return
    driver.get(link)

    xpath = "//*[@id=\"listingContainer\"]/div[*]"

    elements = []

    while len(elements) == 0:
        elements = driver.find_elements_by_xpath(xpath)
        

    for elem in elements:
        print(elem.get_attribute('innerHTML'))
        dic = {}
        dic["Date"] = time
        dic["VenueID"] = venue_name
        dic["DateEvent"] = date
        dic["Quantity"] = elem.find_element_by_tag_name("select").text.split()[0]
        
        print(venue_name)

        price = ""
        while len(price) == 0:
            price = elem.find_element(by=By.TAG_NAME, value="b").get_attribute("textContent")
        print(price)
        dic["Price"] = price[1:]
        row_list.append(dic)

row_list = []

date = datetime.datetime.now()
time = date.strftime("%b %d %I:%M%p")

for i in range(len(links)):
    print(links[i])
    add_page_to_row_list(links[i], row_list, driver, venue_names[i], dates[i], time)

df = pd.DataFrame(row_list)
df.head()

df["Price"].head(100)

server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

<ipython-input-5-56ec26ccbebe>:44: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  links = driver.find_elements_by_xpath(xpath_links)
<ipython-input-5-56ec26ccbebe>:46: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
<ipython-input-5-56ec26ccbebe>:47: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")


['https://www.tickpick.com/buy-yeat-tickets-newport-music-hall-4-9-22-7pm/5076846/', 'https://www.tickpick.com/buy-yeat-tickets-the-fillmore-silver-spring-4-12-22-8pm/5076024/', 'https://www.tickpick.com/buy-yeat-tickets-brooklyn-bowl-philadelphia-4-13-22-8pm/5076025/', 'https://www.tickpick.com/buy-yeat-tickets-the-summit-music-hall-4-15-22-8pm/5076844/', 'https://www.tickpick.com/buy-yeat-tickets-brighton-music-hall-ma-4-17-22-7pm/5076023/', 'https://www.tickpick.com/buy-yeat-tickets-irving-plaza-4-18-22-7pm/5076021/', 'https://www.tickpick.com/buy-yeat-tickets-warsaw-4-19-22-7pm/5076022/', 'https://www.tickpick.com/buy-yeat-tickets-heaven-stage-at-masquerade-ga-4-21-22-6pm/5076845/', 'https://www.tickpick.com/buy-yeat-tickets-the-underground-charlotte-4-24-22-8pm/5076276/', 'https://www.tickpick.com/buy-yeat-tickets-bronze-peacock-at-the-house-of-blues-houston-4-27-22-8pm/5076240/', 'https://www.tickpick.com/buy-yeat-tickets-the-echo-lounge-and-music-hall-4-28-22-8pm/5076241/', 'htt

<ipython-input-5-56ec26ccbebe>:64: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elements = driver.find_elements_by_xpath(xpath)


<label class="sendE"><b>$103</b></label><div class="inputWrapper showFloating"><label class="fieldLabel" for="qDD0">Qty</label><select id="qDD0" class="fieldInput fiL fieldInputSelect" aria-label="Ticket Quantity"><option value="5">5</option><option value="3">3</option><option value="2">2</option><option value="1">1</option></select></div><div class="details"><div class="sout"><span><b>Section GA • </b>Row GA</span></div><label class="score"><b>Lowest Price</b><div class="dealDot" style="background: rgb(208, 208, 208);"></div><div class="dealBG" style="background: rgb(208, 208, 208);"></div></label><a class="notes" data-displaytext="Seats 524 - 528. Mobile Entry Tickets. Must have smart device on hand to enter event."></a><label>E-Transfer</label></div><div class="priceDot" style="background: rgb(240, 229, 5);"></div><div class="star star6">6★</div><div class="border"></div>
The Fillmore Silver Spring - Silver Spring, MD
$103
<label class="sendE"><b>$103</b></label><div class="inputWra

The Fillmore Silver Spring - Silver Spring, MD
$113
<label class="sendE"><b>$120</b></label><div class="inputWrapper showFloating"><label class="fieldLabel" for="qDD9">Qty</label><select id="qDD9" class="fieldInput fiL fieldInputSelect" aria-label="Ticket Quantity"><option value="2">2</option><option value="1">1</option></select></div><div class="details"><div class="sout"><span><b>GA FLOOR • </b>Row 1</span></div><label class="score"><b>–</b> Deal<div class="dealDot" style="background: rgb(208, 208, 208);"></div><div class="dealBG" style="background: rgb(208, 208, 208);"></div></label><a class="notes" data-displaytext="Seats 1 - 2. Mobile Entry Tickets. Must have smart device on hand to enter event."></a><label>E-Transfer</label></div><div class="priceDot" style="background: rgb(240, 222, 8);"></div><div class="star star6">6★</div><div class="border"></div>
The Fillmore Silver Spring - Silver Spring, MD
$120
<label class="sendE"><b>$124</b></label><div class="inputWrapper showFloating

$651
<label class="sendE"><b>$653</b></label><div class="inputWrapper showFloating"><label class="fieldLabel" for="qDD18">Qty</label><select id="qDD18" class="fieldInput fiL fieldInputSelect" aria-label="Ticket Quantity"><option value="2">2</option></select></div><div class="details"><div class="sout"><span><b>Section GAX1 • </b>Row GA5</span></div><label class="score"><b>–</b> Deal<div class="dealDot" style="background: rgb(208, 208, 208);"></div><div class="dealBG" style="background: rgb(208, 208, 208);"></div></label><a class="notes" data-displaytext="Seats 47 - 48. Mobile Entry Tickets. Must have smart device on hand to enter event."></a><label>E-Transfer</label></div><div class="priceDot" style="background: rgb(240, 0, 102);"></div><div class="star star6">6★</div><div class="border"></div>
The Fillmore Silver Spring - Silver Spring, MD
$653
<label class="sendE"><b>$654</b></label><div class="inputWrapper showFloating"><label class="fieldLabel" for="qDD19">Qty</label><select id="qD

KeyboardInterrupt: 

In [11]:
from sqlalchemy.engine.url import URL

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

In [13]:
df.head(100)

,Date,VenueID,DateEvent,Quantity,Price
0,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,
1,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,
2,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,4,76
3,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,79
4,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,4,88
...,...,...,...,...,...
95,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
96,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
97,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
98,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,2,


In [7]:
from sqlalchemy import delete
from selenium import webdriver
from tempfile import mkdtemp
import sqlalchemy
import requests
import numpy as np
import pandas as pd
import datetime
import json
import urllib
from sqlalchemy import create_engine
from selenium_stealth import stealth
import pymysql
import undetected_chromedriver as uc
from sqlalchemy.engine.url import URL
from selenium.webdriver.common.by import By
server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"
f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)
sql = "DELETE FROM test_table WHERE date='Apr 09 07:27PM' OR date='Apr 12 02:27PM'"
 
result = engine.execute(sql)